In [297]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [298]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [299]:
# Compressor Values

Ca4 = 231.16 # m/s
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [300]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [301]:
# Turbine PARAMETERS

htratioturbine = 0.74

In [302]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)

2.2268738031901054


In [303]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0793 m^2
Turbine Inlet Mach Number = 0.25


In [304]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

r4m = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (r4m))
print("r4h = %3.3f m" % (r4h))

r4t = 0.236 m
rm = 0.205 m
r4h = 0.175 m


In [305]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1310 m^2


In [306]:
# constant mean radius
#bladeheight5 = A5/(2*np.pi*rm)
#
#r5t = rm+(bladeheight5/2)
#
#r5h = rm-(bladeheight5/2)


# constant hub radius
r5h = r4h
r5t = np.sqrt(A5/np.pi + (r5h)**2)
r5m = (r5t+r5h)/2

print("r5t = %3.3f m" % (r5t))
print("r5m = %3.3f m" % (r5m))
print("r5h = %3.3f m" % (r5h))

r5t = 0.269 m
r5m = 0.222 m
r5h = 0.175 m


In [307]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

228.4383471772574
1.011914167898574
75.00324865921372
Estimation of 5.1 Stages


In [308]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*r4m*N
U4h = 2*np.pi*r4h*N

In [309]:
stageactual = 7
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

In [310]:
def initialvals4(deltaTocomp,Ca4,To4,po4,htratioturbine):
    deltaToturb = (deltaTocomp*cpa*0.99)/cp

    T4 = To4-((Ca4**2)/(2*cp*1000))

    po4 = 15 # bar
    p4 = po4*(T4/To4)**(gammah/(gammah-1))

    density4 = (p4*100000)/(R*T4)

    A4 = mfthot/(density4*Ca4)

    M4 = Ca4/(np.sqrt(1.333*R*To4))

    r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

    r4h = htratioturbine*r4t

    r4m = (r4t+r4h)/2

    return deltaToturb,r4t,r4h,r4m

In [311]:
def initialvals5(deltaToturb,Ca5,To4,po4,r4h):
    To5 = To4 - deltaToturb

    T5 = To5 - (Ca5**2)/(2*cp*1000)

    po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

    p5 = po5*(T5/To5)**(gammah/(gammah-1))

    density5 = (p5*100000)/(R*T5)

    A5 = mfthot/(density5*Ca5)

    r5h = r4h

    r5t = np.sqrt(A5/np.pi+(r5h)**2)

    r5m = (r5t+r5h)/2

    return r5t,r5h,r5m

In [312]:
def estimatevals(r5h,N,Ca5,bladeloading,deltaToturb):
    U5h = r5h*(2*np.pi*N)

    flow5 = Ca5/U5h

    deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

    stageest = deltaToturb/deltaTos

    return stageest

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [313]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [314]:
def iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca):
    i = 1

    # This runs through the params for each stage and saves them to an array
    while i < stageactual+1:
        if i == 0:
            DOR = 0.5
        if i == 1:
            DOR = 0.55
        if i == 2:
            DOR = 0.6
        if i == 3:
            DOR = 0.7
        else:
            DOR = 0.8

        rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
        rmstage = ((r5m-r4m)/(stageactual-1))*i+r4m
        rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

        Utstage = 2*np.pi*rtstage*N
        Umstage = 2*np.pi*rmstage*N
        Uhstage = 2*np.pi*rhstage*N

        stageTo3 = stagevals[i-1,0,6] - deltaTosact

        pr = ((stageTo3/stagevals[i-1,0,6])**((gammah*polyt)/(gammah-1)))

        # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

        """if i == 1:
            deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
            Cw2t = deltawhirlt - Cw1t
            stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
            stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
            stagealpha1tip = 0
            stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

            V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
            dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
            V2t = V1t*dehallert

            deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
            Cw2m = deltawhirlm - Cw1m
            stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
            stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
            stagealpha1mean = 0
            stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

            V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
            dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
            V2m = V1m*dehallerm

            deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
            Cw2h = deltawhirlh - Cw1h
            stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
            stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
            stagealpha1hub = 0
            stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

            V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
            dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
            V2h = V1h*dehallerh
        else: """
        stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca,DOR)

        stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca,DOR)

        stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca,DOR)

        #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
        stagepo3 = stagevals[i-1,0,14]*pr

        stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rmstage,deltaTosact,stageTo3,Umstage,Ca,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

        i += 1

    return stagevals

In [315]:
deltaTosact = deltaToturb/stageactual
print(deltaTosact)
stagevals = iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca4)

54.42510216525635


In [316]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < len(vals[:,location,4])-1:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 2": vals[:,location,0],"Alpha 3": vals[:,location,1],"Beta 2": vals[:,location,2],"Beta 3": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [317]:
valtable(0,stagevals)

Hub Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,228.4383,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,2220.5749,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9141,13.7120,2.3946,1.0119
Stage 2,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,2166.1498,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9121,12.5066,2.3946,1.0119
Stage 3,41.6073,16.4431,-5.7195,52.0741,0.1748,54.4251,2111.7247,228.4383,231.16,205.2862,68.2232,232.3165,376.0891,0.9100,11.3805,2.3946,1.0119
Stage 4,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,2057.2996,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9077,10.3304,2.3946,1.0119
Stage 5,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,2002.8745,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9054,9.3528,2.3946,1.0119
Stage 6,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,1948.4494,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9029,8.4446,2.3946,1.0119
Stage 7,38.2822,21.5023,-11.2537,54.1145,0.1748,54.4251,1894.0243,228.4383,231.16,182.4423,91.0670,235.6917,394.3585,0.9003,7.6026,2.3946,1.0119


In [318]:
valtable(1,stagevals)

Mean Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,268.5694,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,36.2068,14.6379,-23.9957,55.1923,0.2082,54.4251,2220.5749,272.1237,231.16,169.2255,60.3760,253.0276,404.9580,0.9141,13.7120,1.6875,0.8495
Stage 2,36.0824,14.1284,-24.8839,55.3021,0.2109,54.4251,2166.1498,275.6780,231.16,168.4562,58.1850,254.8165,406.0782,0.9121,12.5066,1.6442,0.8385
Stage 3,40.2437,6.9330,-19.8793,53.0521,0.2136,54.4251,2111.7247,279.2323,231.16,195.6479,28.1085,245.8075,384.5690,0.9100,11.3805,1.6027,0.8278
Stage 4,35.8508,13.1287,-26.6002,55.5287,0.2163,54.4251,2057.2996,282.7867,231.16,167.0293,53.9147,258.5241,408.4149,0.9077,10.3304,1.5626,0.8174
Stage 5,35.7431,12.6383,-27.4293,55.6453,0.2190,54.4251,2002.8745,286.3410,231.16,166.3689,51.8325,260.4385,409.6294,0.9054,9.3528,1.5241,0.8073
Stage 6,35.6406,12.1539,-28.2397,55.7638,0.2218,54.4251,1948.4494,289.8953,231.16,165.7421,49.7840,262.3908,410.8734,0.9029,8.4446,1.4869,0.7974
Stage 7,35.5432,11.6755,-29.0318,55.8841,0.2245,54.4251,1894.0243,293.4496,231.16,165.1478,47.7679,264.3791,412.1460,0.9003,7.6026,1.4511,0.7877


In [319]:
valtable(2,stagevals)

Tip Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,308.7005,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,35.0370,8.7935,-33.6248,56.6746,0.2416,54.4251,2220.5749,315.8091,231.16,162.0824,35.7587,277.6092,420.7551,0.9141,13.7120,1.2529,0.7320
Stage 2,34.9112,7.9203,-34.9553,56.9353,0.2470,54.4251,2166.1498,322.9177,231.16,161.3265,32.1594,282.0402,423.6918,0.9121,12.5066,1.1984,0.7158
Stage 3,39.9564,-1.0778,-30.5360,54.6336,0.2525,54.4251,2111.7247,330.0263,231.16,193.6670,-4.3488,268.3818,399.3755,0.9100,11.3805,1.1473,0.7004
Stage 4,34.7046,6.2304,-37.4484,57.4658,0.2579,54.4251,2057.2996,337.1350,231.16,160.0902,25.2362,291.1697,429.8230,0.9077,10.3304,1.0994,0.6857
Stage 5,34.6221,5.4123,-38.6170,57.7343,0.2633,54.4251,2002.8745,344.2436,231.16,159.5984,21.9010,295.8526,433.0090,0.9054,9.3528,1.0545,0.6715
Stage 6,34.5525,4.6110,-39.7375,58.0043,0.2688,54.4251,1948.4494,351.3522,231.16,159.1841,18.6432,300.6053,436.2700,0.9029,8.4446,1.0122,0.6579
Stage 7,34.4951,3.8258,-40.8123,58.2753,0.2742,54.4251,1894.0243,358.4608,231.16,158.8426,15.4582,305.4217,439.6026,0.9003,7.6026,0.9725,0.6449


In [320]:
#check = False

In [321]:
"""end = 0
Ca = 150
while not check:
    HTratio = 0.3
    while HTratio < .8:
        N = 220
        while N < 250:
            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)
            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)
            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)
            stageactuala = np.ceil(stageest).astype('int32')
            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
            stagevalsa = np.zeros((stageactuala+1,3,17))
            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])
            deltaTosact = deltaToturb/stageactuala
            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)
            
            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78 and stagevalsa[1,0,16] > 0.78 and stagevalsa[stageactuala,2,7] < 290:
                print("Stagecount:"+str(stageactuala))
                print(len(stagevalsa[:,0,4]))
                print(stagevalsa)
                check = True
            N += 1
        HTratio += .1

    Ca += 0.1
    if end > 10000:
        check = True
        print("BAH")
    end += 1

print(HTratio)
print(N)
print(Ca)
print(end)"""

'end = 0\nCa = 150\nwhile not check:\n    HTratio = 0.3\n    while HTratio < .8:\n        N = 220\n        while N < 250:\n            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)\n            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)\n            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)\n            stageactuala = np.ceil(stageest).astype(\'int32\')\n            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff\n            stagevalsa = np.zeros((stageactuala+1,3,17))\n            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])\n            deltaTosact = deltaToturb/stageactuala\n            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)\n            \n            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78